## Baseline

Start with simple data pipeline and make a crappy classifier. This will just the simplest thing that works.

Eliminate all unlabeled profiles, using 80/20 train/test split without consideration for
singleton labels or good stratification of samples, using just composition words as features:

Naive Bayes (default config) 54%/45% train/test

Linear SVM (default config) 96%/51% 

In [1]:
%load_ext autoreload
%autoreload 2

In [31]:
import numpy as np
import profile_classification as pc
raw=pc.load_raw_profile_data()
labeled=pc.get_labeled_profiles(raw)
cut=pc.cut_low_frequencies(labeled,10)
data=pc.make_train_test_data(cut)
#print [(k,v[:10]) for k,v in data.items()]

In [32]:
from mcc_transformers import ItemSelector,EPAText,Rcra
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier


X_train,y_train=data['X_train'],data['y_train']
X_test,y_test=data['X_test'],data['y_test']

print 'There are %d training examples'%len(X_train)

pl=Pipeline([('union',
             FeatureUnion(transformer_list=[('composition',
                                           Pipeline([('get_comp',ItemSelector('Composition')),
                                                    ('vectorize',CountVectorizer(min_df=1,ngram_range=(1,1)))])),
                                            ('rcra',
                                           Pipeline([('get_rcra',ItemSelector('RCRA')),
                                                     ('vectorize', Rcra())])),
                                            ('epaCodes',
                                           Pipeline([('get_codes',ItemSelector('EPA Codes')),
                                                    ('vectorize',EPAText())]))]))])



#vect=CountVectorizer(min_df=1,ngram_range=(1,1))
#svd=TruncatedSVD()
X_v_train=pl.fit_transform(X_train)
#X_v_train=pl.transform(X_train)
X_v_test=pl.transform(X_test)

mNB=MultinomialNB()
svc=LinearSVC()
dt=DecisionTreeClassifier(min_samples_leaf=1)
rf=RandomForestClassifier(n_estimators=100)
knn=KNeighborsClassifier(weights='distance',n_neighbors=3)
for clf in (mNB,svc,dt,rf,knn): 
    clf.fit(X_v_train,y_train)
    y_pred=clf.predict(X_v_test)
    print type(clf),np.sum(y_pred==y_test)/float(len(y_test))
    #print classification_report(y_test,y_pred)
    

There are 3088 training examples
<class 'sklearn.naive_bayes.MultinomialNB'> 0.493523316062
<class 'sklearn.svm.classes.LinearSVC'> 0.551813471503
<class 'sklearn.tree.tree.DecisionTreeClassifier'> 0.507772020725
<class 'sklearn.ensemble.forest.RandomForestClassifier'> 0.591968911917
<class 'sklearn.neighbors.classification.KNeighborsClassifier'> 0.520725388601


In [5]:
vect=CountVectorizer(min_df=1,ngram_range=(1,1))
r_w=raw.dropna(subset=['Composition'])
r_w=r_w['Composition']
vect.fit(r_w)


CountVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern=u'(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [8]:
print len(vect.get_feature_names())
print vect.get_feature_names()[-200:]

9258
[u'violent', u'violet', u'virgin', u'virus', u'viscor', u'viscosity', u'visqueen', u'vitamin', u'vitamins', u'vitavax', u'vitreous', u'vitrious', u'vm', u'voc', u'vocs', u'vodka', u'volatile', u'volume', u'vp', u'vr', u'vulcanized', u'vulcanizing', u'vydate', u'wa', u'wafarin', u'wafer', u'wall', u'walled', u'wararin', u'ward', u'warf', u'warfar', u'warfarin', u'warfrin', u'wart', u'was', u'wash', u'washed', u'washer', u'washers', u'washout', u'waste', u'wastes', u'wat', u'watches', u'wate', u'water', u'waterbased', u'watermark', u'waterproofing', u'wax', u'waxes', u'waxy', u'wc', u'wd', u'weak', u'weapons', u'weather', u'weed', u'weeds', u'weevil', u'weigh', u'weight', u'weld', u'welding', u'well', u'were', u'west', u'wet', u'wetted', u'wetting', u'wh15sl3', u'what', u'wheat', u'when', u'which', u'while', u'whipped', u'whiskey', u'white', u'wi', u'wick', u'wide', u'will', u'windshield', u'wine', u'wipe', u'wipers', u'wipes', u'wire', u'wires', u'wiring', u'witco', u'with', u'with

In [6]:
print pl.named_steps['union'].transformer_list[2][1].named_steps['vectorize'].get_feature_names()

[u'd001', u'd002', u'd003', u'd004', u'd005', u'd006', u'd007', u'd008', u'd009', u'd010', u'd011', u'd012', u'd013', u'd014', u'd015', u'd016', u'd017', u'd018', u'd019', u'd020', u'd021', u'd022', u'd023', u'd024', u'd025', u'd026', u'd027', u'd028', u'd029', u'd030', u'd031', u'd032', u'd033', u'd034', u'd035', u'd036', u'd037', u'd038', u'd039', u'd040', u'd041', u'd042', u'd043', u'f001', u'f002', u'f003', u'f004', u'f005', u'f006', u'f007', u'f008', u'f009', u'f010', u'f011', u'f012', u'f019', u'f024', u'f025', u'f032', u'f034', u'f035', u'f037', u'f038', u'f039', u'k001', u'k002', u'k003', u'k004', u'k005', u'k006', u'k007', u'k008', u'k009', u'k010', u'k011', u'k013', u'k014', u'k015', u'k016', u'k017', u'k018', u'k019', u'k020', u'k021', u'k022', u'k027', u'k028', u'k029', u'k030', u'k031', u'k033', u'k034', u'k035', u'k048', u'k049', u'k050', u'k051', u'k052', u'k061', u'k083', u'k086', u'k087', u'k094', u'k095', u'k096', u'k103', u'k104', u'k141', u'k142', u'k143', u'k144', 